In this notebook, me and my team, we are going to scrape an ecommerce website. We'll get into each individual product page and retrieve our information from there. This is the website we are going to scrape <a href="https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt">Cdiscount</a>  – it's an French online shop.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Maintenant, nous allons définir l'URL de base de la page principale car nous en aurons besoin lorsque nous construirons nos URL pour chacun des produits individuels.

De plus, nous enverrons un agent utilisateur sur chaque requête HTTP, car si nous avons effectuez une requête GET à l'aide de requests , l'agent utilisateur est Python par défaut, ce qui peut être bloqué.

Donc, pour remplacer cela, nous allons déclarer une variable qui stockera notre user-agent.

In [ ]:
baseurl = "https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [ ]:
k = requests.get(baseurl, headers=headers).text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"cPdtItem jsPdtItem"})
soup


In [ ]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep

In [ ]:
baseurl="https://www.amazon.com/dp/B07YDGHDFQ/ref=sspa_dk_detail_1?psc=1&pd_rd_i=B07YDGHDFQ&pd_rd_w=V8Vij&pf_rd_p=0c758152-61cd-452f-97a6-17f070f654b8&pd_rd_wg=xTOAJ&pf_rd_r=P8H8N568MSXRHR7H4DRH&pd_rd_r=bb5f00a6-6961-4b95-8810-dbb98dff2669&s=apparel&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzVERWRVJHUkRNOEpNJmVuY3J5cHRlZElkPUEwNDQyOTIxM0JaODZZMFI4SjhFOCZlbmNyeXB0ZWRBZElkPUEwMTE3NDc3MUhKRUZONFJVTzRaUSZ3aWRnZXROYW1lPXNwX2RldGFpbCZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="

In [ ]:
baseurl="https://www.amazon.com/Cass-Creations-iPhone-Bumper-Protective/dp/B096ZCT3N3/ref=sr_1_14_sspa?pd_rd_r=30305d08-93c6-4f70-aca0-5bdcd0619f0a&pd_rd_w=V3rlu&pd_rd_wg=cnq1i&pf_rd_p=f5c158e1-98f7-4998-94b8-d7306c066086&pf_rd_r=Q0RJ0DQSNB5EM8EWZX6G&qid=1649545449&refinements=p_72%3A2491149011&s=wireless&sr=1-14-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyMlpGS1o2MU1UNE5EJmVuY3J5cHRlZElkPUEwMzI2MjEyTU1QNVJHU1NDVlpVJmVuY3J5cHRlZEFkSWQ9QTA4MjMxNzAxTlVYRjRSVTBTQlpRJndpZGdldE5hbWU9c3BfbXRmX2Jyb3dzZSZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="

In [28]:
from ast import literal_eval
import pandas as pd
 
with open(r'D:\INPT2\Data mining\DataMining-Project\web Scraping\output.json',"r") as f:
    data = f.readlines()
    data = [literal_eval(x.strip()) for x in data]

print(data)
pd.DataFrame(data)

ValueError: malformed node or string on line 1: <ast.Name object at 0x000002069B44AE30>